
# Bitcoin News Sentiment Analysis API Demo

This notebook demonstrates how to use the Bitcoin News Sentiment Analysis API to fetch news articles, analyze their sentiment, and store the results.



## 1. Setup Environment

First, let's load environment variables and import the required modules.


In [1]:

# Import necessary libraries
import pandas as pd
from dotenv import load_dotenv
import os
from pathlib import Path

# Import our utility functions
from bitcoin_utils import (
    Config, fetch_bitcoin_news, classify_sentiment,
    process_news_articles, make_clickable, create_sentiment_pie_chart
)

# Load environment variables from .env file
load_dotenv()

# Check that we have the required API key
if not os.getenv("NEWS_API_KEY"):
    raise ValueError("NEWS_API_KEY not found in .env file")
    
print("Environment loaded successfully")


Environment loaded successfully



## 2. Initialize Configuration

Create a configuration object that will hold all our settings and API keys.


In [2]:

# Initialize the configuration
config = Config.load()

# Create data directory if it doesn't exist
config.DATA_DIR.mkdir(exist_ok=True)

print(f"Using LLM provider: {config.PROVIDER}")
print(f"Data directory: {config.DATA_DIR}")


Using LLM provider: together_ai
Data directory: /Users/sandeep/Desktop/Project25/Data605/data



## 3. Fetch Bitcoin News

Now let's fetch some recent Bitcoin news articles.


In [3]:

# Fetch latest Bitcoin news (last 3 days, 5 articles)
articles = fetch_bitcoin_news(config, page_size=5, days_back=3)

# Convert to DataFrame for easier viewing
df_articles = pd.DataFrame([
    {
        'title': article.get('title'),
        'publishedAt': article.get('publishedAt'),
        'source': article.get('source', {}).get('name'),
        'url': article.get('url')
    } for article in articles
])

# Display the articles
df_articles.head()


Fetching Bitcoin news from 2025-05-13 to 2025-05-16...


,title,publishedAt,source,url
0,Bitcoin Repeats April Playbook – Watch This Ra...,2025-05-16T23:30:55Z,Bitcoinist,https://bitcoinist.com/bitcoin-repeats-april-p...
1,Wisconsin Pension Fund Sold IBIT Before Trade ...,2025-05-16T23:30:00Z,Etf.com,http://www.etf.com/sections/news/wisconsin-pen...
2,Moody’s historic downgrade of US credit system...,2025-05-16T23:21:36Z,CryptoSlate,https://cryptoslate.com/moodys-historic-downgr...
3,How Silicon Valley's influence in Washington b...,2025-05-16T22:15:09Z,TechCrunch,https://techcrunch.com/2025/05/16/how-silicon-...
4,Crypto Bulls Just Got Their Macro Wake-Up Call...,2025-05-16T22:00:53Z,newsBTC,http://www.newsbtc.com/news/crypto-bulls-just-...



## 4. Analyze Sentiment on Individual Headlines

Let's analyze the sentiment of a few headlines to demonstrate the API.


In [4]:

# Take the first 3 articles for demonstration
demo_articles = articles[:3]

# Analyze sentiment for each article
results = []
for article in demo_articles:
    title = article.get('title')
    content = article.get('description') or article.get('content', '')
    
    # Skip if no content
    if not content:
        print(f"No content for: {title}")
        continue
    
    # Analyze sentiment
    sentiment, cost = classify_sentiment(content, config)
    
    # Store result
    results.append({
        'headline': title,
        'content_preview': content[:100] + '...',
        'sentiment': sentiment,
        'score': 1 if sentiment.lower() == 'positive' else (-1 if sentiment.lower() == 'negative' else 0)
    })
    print(f"[{sentiment}] {title}")

# Display results
pd.DataFrame(results)


[Pos] Bitcoin Repeats April Playbook – Watch This Range for Confirmation
[Ne] Wisconsin Pension Fund Sold IBIT Before Trade Clash
[Ne] Moody’s historic downgrade of US credit system could add fuel to Bitcoin safe-haven status


,headline,content_preview,sentiment,score
0,Bitcoin Repeats April Playbook – Watch This Ra...,Bitcoin is trading just 5% below its all-time ...,Pos,0
1,Wisconsin Pension Fund Sold IBIT Before Trade ...,- The Wisconsin pension fund divested its enti...,Ne,0
2,Moody’s historic downgrade of US credit system...,Moody’s downgraded the United States’ long-ter...,Ne,0



## 5. Process All Articles and Save to CSV

Now let's process all articles in a batch and save the results to a CSV file.


In [5]:

# Process all articles
df_processed = process_news_articles(config, page_size=10, days_back=3)

# Display summary
print(f"\nProcessed {len(df_processed)} articles")
print("\nSentiment distribution:")
print(df_processed['sentiment'].value_counts())

# Save to CSV
output_file = config.DATA_DIR / "bitcoin_demo_results.csv"
df_processed.to_csv(output_file, index=False)
print(f"\nResults saved to {output_file}")

# Show the results
df_processed.head()


Fetching Bitcoin news from 2025-05-13 to 2025-05-16...
Fetched 10 articles
[CACHED] [Pos] Bitcoin Repeats April Playbook – Watch This Range for Confirmation...
[CACHED] [Ne] Wisconsin Pension Fund Sold IBIT Before Trade Clash...
[CACHED] [Ne] Moody’s historic downgrade of US credit system could add fuel to Bitcoin safe-ha...
[CACHED] [Ne] How Silicon Valley's influence in Washington benefits the tech elite | TechCrunc...
[CACHED] [Ne] Crypto Bulls Just Got Their Macro Wake-Up Call: Here’s Why...
[CACHED] [Neg] Bitcoin Double-Top Pattern Says Bull Market Is Over, Is A Crash Coming?...
[CACHED] [Ne] SEC bitcoin hack case gets Alabama man sentenced to 14 months in prison...
[CACHED] [Ne] Alabama man sentenced to 14 months in prison for role in SEC bitcoin announcemen...
[CACHED] [Pos] Basel Medical Group to add $1B in Bitcoin to treasury amid falling share prices...
[CACHED] [Pos] Coinbase Stock Is Ripping Despite Hack Fears—Here's Why...
Articles processed: 10 (New: 0, Cached: 10)
Total 

,publishedAt,headline,sentiment,score,url,cost,cached
0,2025-05-16 23:30:55+00:00,Bitcoin Repeats April Playbook – Watch This Ra...,Pos,0,https://bitcoinist.com/bitcoin-repeats-april-p...,0.0,True
1,2025-05-16 23:30:00+00:00,Wisconsin Pension Fund Sold IBIT Before Trade ...,Ne,0,http://www.etf.com/sections/news/wisconsin-pen...,0.0,True
2,2025-05-16 23:21:36+00:00,Moody’s historic downgrade of US credit system...,Ne,0,https://cryptoslate.com/moodys-historic-downgr...,0.0,True
3,2025-05-16 22:15:09+00:00,How Silicon Valley's influence in Washington b...,Ne,0,https://techcrunch.com/2025/05/16/how-silicon-...,0.0,True
4,2025-05-16 22:00:53+00:00,Crypto Bulls Just Got Their Macro Wake-Up Call...,Ne,0,http://www.newsbtc.com/news/crypto-bulls-just-...,0.0,True



## 6. Read Results from CSV

Let's read back the results from the CSV file to demonstrate persistence.


In [6]:

# Read from CSV
df_from_csv = pd.read_csv(output_file)

# Convert date column back to datetime
df_from_csv['publishedAt'] = pd.to_datetime(df_from_csv['publishedAt'])

print(f"Read {len(df_from_csv)} articles from CSV")
df_from_csv.head()


Read 10 articles from CSV


,publishedAt,headline,sentiment,score,url,cost,cached
0,2025-05-16 23:30:55+00:00,Bitcoin Repeats April Playbook – Watch This Ra...,Pos,0,https://bitcoinist.com/bitcoin-repeats-april-p...,0.0,True
1,2025-05-16 23:30:00+00:00,Wisconsin Pension Fund Sold IBIT Before Trade ...,Ne,0,http://www.etf.com/sections/news/wisconsin-pen...,0.0,True
2,2025-05-16 23:21:36+00:00,Moody’s historic downgrade of US credit system...,Ne,0,https://cryptoslate.com/moodys-historic-downgr...,0.0,True
3,2025-05-16 22:15:09+00:00,How Silicon Valley's influence in Washington b...,Ne,0,https://techcrunch.com/2025/05/16/how-silicon-...,0.0,True
4,2025-05-16 22:00:53+00:00,Crypto Bulls Just Got Their Macro Wake-Up Call...,Ne,0,http://www.newsbtc.com/news/crypto-bulls-just-...,0.0,True



## 7. Visualize Sentiment Distribution

Finally, let's visualize the sentiment distribution using the helper function.


In [7]:

# Create pie chart
fig = create_sentiment_pie_chart(df_from_csv)
fig.show()



## Summary

This notebook demonstrated how to:
1. Load configuration from environment variables
2. Fetch Bitcoin news articles
3. Analyze sentiment on individual articles
4. Process multiple articles and save results to CSV
5. Read results back from CSV
6. Visualize sentiment distribution

The caching mechanism ensures that previously analyzed articles are not processed again, saving API tokens.
